In [2]:
# these three lines swap the stdlib sqlite3 lib with the pysqlite3 package
__import__('pysqlite3')
import sys
sys.modules['sqlite3'] = sys.modules.pop('pysqlite3')

from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import UnstructuredFileLoader # UnstructuredFileLoader는 pdf, txt, docx를 다 열 수 있음
from langchain.text_splitter import RecursiveCharacterTextSplitter, CharacterTextSplitter
from langchain.vectorstores.chroma import Chroma #vector stores사용(여러 vector stores중 chroma는 오픈소스임)
from langchain.embeddings import OpenAIEmbeddings, CacheBackedEmbeddings
from langchain.storage import LocalFileStore
from langchain.chains import RetrievalQA

llm = ChatOpenAI()

cache_dir = LocalFileStore("../.cache/") # embedding한 vector를 캐싱하기 위해 캐쉬 디렉토리 설정

splitter = CharacterTextSplitter.from_tiktoken_encoder(
    separator="\n",
    chunk_size=600,
    chunk_overlap=100,
) # 특정 문자열을 기준으로 끊어줌

loader = UnstructuredFileLoader("../../files/chapter_one.docx")

docs = loader.load_and_split(text_splitter=splitter)

embeddings = OpenAIEmbeddings()

# cache embedding을 설정함으로써 embedding을 할 때, 캐시에 embedding이 이미 존재하는지 확인하고
# 없으면 vector store를 호출할때 문서들과 OpenAIEmbeddings를 사용하게 됨
cached_embeddings = CacheBackedEmbeddings.from_bytes_store(
    embeddings, cache_dir    
)


vectorstore = Chroma.from_documents(docs, cached_embeddings)

chain = RetrievalQA.from_chain_type(
    llm = llm,
    chain_type="stuff",
    retriever=vectorstore.as_retriever()
)

"""
 [chain_type] RetreivalQA를 사용할때 장점 중 하나가 chain_type을 쉽게 바꿀 수 있다는 것임
  - refine : chain type을 "refine" 으로 할 경우 질문->답변->정제된 질문->답변 -> ... 으로 더 좋은 답변을 받을 수 있지만
    체인 내부에서 질문->답변을 반복하므로 stuff보다 돈이 많이 소모됨
  - map_reduce : query를 입력하면 document들을 받아서 개별적으로 추출한 뒤 추출한 내용들을 LLM에 전달함
  - map_rerank : query를 입력하면 연관된 document를 받아서 각 document를 통해 답변을 생성하고 각 답변에 점수를 매김. 마지막으로 가장 높은 점수를 획득한 답변과 그 점수를 함께 반환함

 [retriever]
   - retriever는 여러 장소에서 document들을 가져오는 클래스의 interface임 (vector store보다 더 일반화된 형태)
"""
chain.run("Describe Victory Mansions") # chain에 질문query를 보내면 retriever가 document들을 검색한뒤 관련된 문서를 찾아서 llm(위 코드에서는 ChatOpenAI)에 질문과 문서를 함께 보내고 답변을 받아옴



'Victory Mansions is a building where Winston Smith resides. It has glass doors at the entrance and is located in a cold and windy area. The hallway of the building has a strong smell of boiled cabbage and old rag mats. On one end of the hallway, there is a large colored poster of an enormous face, about a meter wide, depicting a man in his forties with a heavy black mustache. The face has ruggedly handsome features and is accompanied by the caption "BIG BROTHER IS WATCHING YOU." The building has seven floors, and Winston\'s flat is located on the seventh floor. The lift in the building is often out of service, so Winston usually takes the stairs. Inside his flat, there is a telescreen, a voice-operated device that cannot be fully turned off, constantly broadcasting news and propaganda. The flat is sparsely furnished, and Winston\'s physical appearance is described as smallish and frail, with fair hair and rough skin.'